In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor

In [2]:
market = Market()
market.connect()
dates = market.retrieve_date_range()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
stuff = pd.to_datetime(dates["date"])

In [5]:
start = (stuff.max() + timedelta(days=1)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2021-11-06 2021-11-09


In [13]:
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()[:1]):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        data = TiingoExtractor.extract(ticker,start,end)
        data["ticker"] = ticker
#         market.store("prices",data)
    except Exception as e:
        print(str(e))
market.disconnect()

 16%|████████████████████▋                                                                                                             | 80/504 [00:50<04:28,  1.58it/s]

project_market prices documents must be a non-empty list


 61%|██████████████████████████████████████████████████████████████████████████████                                                   | 305/504 [03:16<02:16,  1.46it/s]

project_market prices documents must be a non-empty list


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 501/504 [05:20<00:01,  1.62it/s]

...

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 503/504 [05:21<00:00,  1.67it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 504/504 [05:21<00:00,  1.57it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


In [14]:
data

In [20]:
analysis = []
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_data(ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        periods = ["year","quarter","month","week"]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(str(e))
market.disconnect()

 13%|████████████████▋                                                                                                                 | 65/505 [02:06<14:05,  1.92s/it]

'date'


 16%|████████████████████▎                                                                                                             | 79/505 [02:33<13:33,  1.91s/it]

'date'


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 502/505 [16:20<00:05,  1.92s/it]

...

'date'


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 504/505 [16:24<00:01,  1.91s/it]

'date'


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [16:26<00:00,  1.95s/it]

'date'


In [16]:
window

,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor,ticker,year,quarter,week,month
7014,2021-11-08 00:00:00+00:00,181.54,183.43,180.27,182.66,1847631,181.54,183.43,180.27,182.66,1847631,0.0,1.0,MMM,2021,4,45,11


In [21]:
a = pd.DataFrame(analysis).merge(sp5.rename(columns={"Symbol":"ticker"})[["ticker","Security","GICS Sector"]],on="ticker",how="left")
a.to_csv("analysis.csv")

In [24]:
period = "day"
a.sort_values(f"{period}_gain",ascending=False).head(50)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector
7,0.626869,0.465691,0.199074,0.000000,0.101364,AMD,Advanced Micro Devices,Information Technology
497,0.344633,0.097032,0.000000,0.000000,0.095497,XYL,Xylem,Industrials
472,-0.196566,0.079412,0.062229,0.000000,0.069192,VTRS,Viatris,Health Care
202,0.459001,0.200670,0.028698,0.000000,0.064525,FCX,Freeport-McMoRan,Materials
96,0.653339,0.022898,0.085952,0.000000,0.056419,CF,CF Industries,Materials
475,0.412953,0.189777,0.093537,0.000000,0.049475,VMC,Vulcan Materials,Materials
496,0.460437,0.369658,0.124104,0.000000,0.047857,XLNX,Xilinx,Information Technology
225,0.653088,0.064949,0.021516,0.000000,0.046760,HES,Hess Corporation,Energy
340,0.265298,0.012520,0.020907,0.000000,0.044812,NLOK,NortonLifeLock,Information Technology
195,1.365023,0.423023,0.122563,0.000000,0.044583,F,Ford,Consumer Discretionary


In [26]:
a.groupby("GICS Sector").mean().sort_values("day_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Energy,0.715643,0.105469,0.011763,0.000391,0.011734
Materials,0.304761,0.089748,0.041886,0.000000,0.010861
Health Care,0.195446,0.029519,-0.003658,0.000000,0.007233
Information Technology,0.280510,0.084475,0.036330,0.000000,0.007039
Industrials,0.300986,0.078082,0.021460,0.000000,0.004466
Financials,0.411122,0.055445,0.000999,0.000000,0.003218
Real Estate,0.453844,0.068605,0.012873,0.000000,-0.002964
Consumer Discretionary,0.324739,0.089367,0.026670,0.000000,-0.004521
Consumer Staples,0.082970,0.029442,0.015684,0.000000,-0.006002
